In [8]:
# city scapes
#
import os
import imageio
from pathlib import Path

In [7]:
from PIL import Image


In [ ]:
## remove non essential files

In [2]:
directory= "../cityscapes_pretraiement/gtFine/train/"
for f in os.listdir(directory):
    filename = directory+""+f
    if not f.endswith("_labelTrainIds.png"):
        os.remove(filename)

In [3]:
directory= "../cityscapes_pretraiement/gtFine/val/"
for f in os.listdir(directory):
    filename = directory+""+f
    if not f.endswith("_labelTrainIds.png"):
        os.remove(filename)

In [ ]:
## resize

In [11]:
directory= "../cityscapes_pretraiement/gtFine/train/"
for f in os.listdir(directory):
    filename = directory+""+f
    img = Image.open(filename)
    img = img.resize((128,128), Image.ANTIALIAS)
    img.save(directory+""+f)

In [12]:
directory= "../cityscapes_pretraiement/gtFine/val/"
for f in os.listdir(directory):
    filename = directory+""+f
    img = Image.open(filename)
    img = img.resize((128,128), Image.ANTIALIAS)
    img.save(directory+""+f)

In [15]:
directory= "../cityscapes_pretraiement/leftImg8bit/train/"
for f in os.listdir(directory):
    filename = directory+""+f
    img = Image.open(filename)
    img = img.resize((128,128), Image.ANTIALIAS)
    img.save(directory+""+f)

In [16]:
directory= "../cityscapes_pretraiement/leftImg8bit/val/"
for f in os.listdir(directory):
    filename = directory+""+f
    img = Image.open(filename)
    img = img.resize((128,128), Image.ANTIALIAS)
    img.save(directory+""+f)